# Strands Agents クイックスタートガイド

Strands Agentsは、AWSサービスと連携し複雑なタスクを実行できるAIエージェントを構築するための強力なフレームワークです。このクイックスタートガイドでは、最初のStrandsエージェントの作成方法を紹介します。

## 前提条件

- Python 3.10 以上
- 適切な権限が設定されたAWSアカウント
- Pythonプログラミングの基礎知識

さっそく始めましょう！

In [ ]:
# Install Strands using pip

!pip install strands-agents strands-agents-tools


## はじめてのエージェントを作成

エージェントを構成する主要なコンポーネントについて概要を確認しましょう。

### 1. シンプルなエージェントを作成

デフォルトのモデルプロバイダーである[Amazon Bedrock](https://aws.amazon.com/bedrock/)と、デフォルトモデルClaude 3.7 Sonnet（ご利用のAWSリージョン）を使ったエージェントを作成します。エージェントはローカル環境で動作しますが、Amazon BedrockのモデルはAWSアカウント上で実行され、エージェントはクラウド上のモデルを呼び出します。アーキテクチャは以下の通りです：

<div style="text-align:center">
    <img src="images/simple_agent.png" width="75%" />
</div>

In [ ]:
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 

from strands import Agent
# Initialize your agent
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt="You are a helpful assistant that provides concise responses."
)

# Send a message to the agent
response = agent("Hello! Tell me a joke.")
# 日本語で送信
# response = agent("こんにちは！ジョークを教えて。")

### 2. エージェントにツールを追加

[strands-agents-tools](https://github.com/strands-agents/tools) リポジトリには、インポート可能な組み込みツールが用意されています。また、`@tool`デコレーターを使ってカスタムツールを作成することもできます。組み込みツールやカスタムツールを組み合わせてエージェントを作成できます。例えば、電卓の組み込みツールと天気を取得するカスタムツールを追加すると、以下のようなアーキテクチャになります：
<div style="text-align:center">
    <img src="images/agent_with_tools.png" width="75%" />
</div>

このアーキテクチャを実装すると、次のようになります:

In [ ]:
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.")

response = agent("What is the weather today?")
# 日本語で送信
# response = agent("今日はどんな天気ですか？")

print(response)

### ツールを直接呼び出す

一部の用途では、ツールを直接呼び出すことが重要です。たとえば、ツールのデバッグや、エージェントの知識を事前に顧客情報で埋め込む場合、またはツール内で別のツールを使いたい場合などです。Strandsでは、エージェントの`tool`メソッドの後にツール名を続けて呼び出すことができます。

In [ ]:
# Alternatively, you can invoke the tool directly like so:
agent.tool.calculator(expression="sin(x)", mode="derive", wrt="x", order=2)


### 3. ログレベルとフォーマットの変更

Strands SDKは、Python標準の`logging`モジュールを利用して動作状況を可視化します。

Strands Agents SDKのロギングはシンプルな設計です：

1. **モジュール単位のロガー**：各モジュールはlogging.getLogger(__name__)で独自のロガーを作成し、Pythonの階層的なロギングのベストプラクティスに従っています。
2. **ルートロガー**：SDK内のすべてのロガーは「strands」ルートロガーの子となり、SDK全体のロギング設定が容易です。
3. **デフォルト動作**：SDKはデフォルトでハンドラやログレベルを設定しないため、アプリケーション側のロギング設定と統合できます。

Strands Agents SDKのロギングを有効化したい場合や、デバッグ時にログレベルやフォーマットを変更したい場合は、以下のように設定できます：

In [ ]:
import logging
from strands import Agent

# Enables Strands debug log level
logging.getLogger("strands").setLevel(logging.DEBUG) # or logging.INFO

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

agent = Agent(model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")  # Optional: Specify the model ID
agent("Hello!")


### 4. モデルプロバイダー

デフォルトのモデルプロバイダーは[Amazon Bedrock](https://aws.amazon.com/bedrock/)で、デフォルトモデルはご利用中のAWS環境リージョンのClaude 3.7 Sonnetです。

Amazon Bedrockで別のモデルを利用したい場合は、モデルID文字列を直接指定できます：

In [ ]:
from strands import Agent

agent = Agent(model="anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)


モデル構成をさらに細かく制御したい場合は、`BedrockModel`プロバイダーインスタンスを作成できます：

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name='us-west-2',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)


利用可能なモデルプロバイダーの詳細は、[モデルプロバイダー クイックスタートページ](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers) をご覧ください。


**おめでとうございます！これでStrandsを使ったシンプルなエージェントの作り方を学びました！**

## 【オプション】タスク特化型エージェント RecipeBot 🍽️ を作ってみよう

実践的なタスク特化型エージェントの例として、レシピを提案したり料理に関する質問に答える「RecipeBot」を作成します。さっそく始めましょう！

以下のようなものを作成します：

<div style="text-align:center">
    <img src="images/interactive_recipe_agent.png" width="75%" />
</div>

In [ ]:
# Install the required packages
%pip install ddgs # Also install strands-agents strands-agents-tools if you haven't already

In [ ]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)

# Define a websearch tool
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


In [ ]:
# Create a recipe assistant agent
recipe_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt="""You are RecipeBot, a helpful cooking assistant.
    Help users find recipes based on ingredients and answer cooking questions.
    Use the websearch tool to find recipes when users mention ingredients or to
    look up cooking information.""",
    # Import the websearch tool we created above
    tools=[websearch],
)

In [ ]:
response = recipe_agent("Suggest a recipe with chicken and broccoli.")

# Other examples:
# response = recipe_agent("How do I cook quinoa?")
# response = recipe_agent("How can I substitute white wine in shrimp pasta?")
# response = recipe_agent("What are the health benefits of asparagus?")
print(response)

In [ ]:
# 日本語
response = recipe_agent("鶏肉とブロッコリーを使ったレシピを提案してください。")

# 他の例:
# response = recipe_agent("キヌアの調理方法を教えてください。")
# response = recipe_agent("エビのパスタで白ワインの代用は何ができますか？")
# response = recipe_agent("アスパラガスの健康効果を教えてください。")
print(response)

#### 以上です！たった数行のコードでツール付きのユースケースエージェントが完成しました🥳。

より詳しいクイックスタートガイドは[Strandsドキュメント](https://strandsagents.com/0.1.x/user-guide/quickstart/)をご覧ください。

### 【オプション】RecipeBotをCLIで実行する

SageMaker Studioなどのターミナルから、`02_simple_interactive_usecase/recipe_bot.py`のPythonスクリプトを使ってエージェントを対話モードで実行できます。これにより、コマンドライン上でメッセージを送信し、応答を受け取ることができます。

以下のコマンドを実行してください：

```cli
cd samples/01-tutorials/01-fundamentals/01-first-agent/02-simple-interactive-usecase/
pip install -r requirements.txt
python recipe_bot.py
```

これで、コマンドラインインターフェース（CLI）を通じてボットと会話できます。